# MIDI music generation with GAN

In [ ]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
!unzip 'maestro-v3.0.0-midi.zip'
!rm 'maestro-v3.0.0-midi.zip'
dataset_path = "maestro-v3.0.0"

--2025-08-11 08:54:29--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.130.207, 74.125.68.207, 142.250.4.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.130.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: ‘maestro-v3.0.0-midi.zip’

maestro-v3.0.0-midi 100%[===================>]  55.71M  17.3MB/s    in 3.6s    

2025-08-11 08:54:33 (15.5 MB/s) - ‘maestro-v3.0.0-midi.zip’ saved [58416533/58416533]

Archive:  maestro-v3.0.0-midi.zip
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_08_R1_2004_01-02_ORIG_MID--AUDIO_08_R1_2004_01_Track01_wav.midi  
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_09_R1_2004_05_ORIG_MID--AUDIO_09_R1_2004_06_Track06_wav.midi  
  inflating: maestro-v3.0.0/2004/MIDI-Unprocessed_XP_14_R1_2004_01-03_ORIG_MID--AUDIO_14_R1_2004_01_Track

In [1]:
!pip install miditok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.0/159.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.9 MB/s eta 0:00:00


In [2]:
!pip install -q gdown

In [3]:
import os
import shutil
from pathlib import Path
import os
import numpy as np
from IPython.display import Audio, display
from miditok import REMI, TokenizerConfig
from pathlib import Path
from miditok.data_augmentation import augment_dataset
from miditok.utils import split_files_for_training
from random import shuffle
import json
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [ ]:
destination_folder = "dataset_midi"
os.makedirs(destination_folder, exist_ok=True)

midi_paths = list(Path(dataset_path).resolve().glob("**/*.mid")) + \
             list(Path(dataset_path).resolve().glob("**/*.midi"))

print(f"Trovati {len(midi_paths)} file MIDI.")

for midi_path in midi_paths:
    filename = os.path.basename(midi_path)
    destination_path = os.path.join(destination_folder, filename)
    shutil.copy2(midi_path, destination_path)

print("Copia completata in 'dataset_midi'.")

# Convert to relative paths
midi_paths = [Path(os.path.join(destination_folder, os.path.basename(p))) for p in midi_paths]

Trovati 1276 file MIDI.
Copia completata in 'dataset_midi'.


## Tokenizzazione

Setup iniziale

In [ ]:
BEAT_RES = {(0, 1): 12, (1, 2): 4, (2, 4): 2, (4, 8): 1}
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": BEAT_RES,
    "num_velocities": 3,
    "special_tokens": ["BOS", "EOS"],
    "use_chords": True,
    "use_rests": True,
    "use_tempos": True,
    "num_tempos": 8,
    "tempo_range": (50, 200),
}

Carica file MIDI

In [ ]:
midi_dir = Path(destination_folder)
midis = list(midi_dir.glob("**/*.mid")) + list(midi_dir.glob("**/*.midi"))

if not midis:
    raise FileNotFoundError("Nessun file MIDI trovato in 'dataset_midi'.")

Train o carica il tokenizer

In [ ]:
config = TokenizerConfig(**TOKENIZER_PARAMS)

tokenizer = REMI(config)

vocab_size = 5000
tokenizer.train(vocab_size=vocab_size, files_paths=midis)
processed = [Path(f"{s}") for s in midis]
print(len(processed))

1276


In [ ]:
tokenizer.save("tokenizerMIDI")

Tokenizza i file MIDI

dataset preparation

In [ ]:
# Assicurati che la directory base sia un Path assoluto
base_dir = Path("/content/dataset_midi").resolve()

# Ottieni tutti i file MIDI come path assoluti
midis = list(base_dir.glob("**/*.mid")) + list(base_dir.glob("**/*.midi"))
midis = [midi.resolve() for midi in midis]

# Suddivisione in train/valid/test
total_num_files = len(midis)
num_files_valid = round(total_num_files * 0.15)
num_files_test = round(total_num_files * 0.15)

shuffle(midis)

midi_paths_valid = midis[:num_files_valid]
midi_paths_test = midis[num_files_valid:num_files_valid + num_files_test]
midi_paths_train = midis[num_files_valid + num_files_test:]

# Chunk e augment
for files_paths, subset_name in (
    (midi_paths_train, "train"),
    (midi_paths_valid, "valid")
):
    subset_chunks_dir = Path(f"Maestro_{subset_name}")

    split_files_for_training(
        files_paths=files_paths,
        tokenizer=tokenizer,
        save_dir=subset_chunks_dir,
        max_seq_len=1024,
        num_overlap_bars=2,
    )

    augment_dataset(
        subset_chunks_dir,
        pitch_offsets=[-12, 12],
        velocity_offsets=[-3, 3],
        duration_offsets=[-0.5, 0.5],
    )

# Rileggi i file MIDI già chunkati e augmentati per ciascun subset
midi_paths_train = list(Path("Maestro_train").glob("**/*.mid")) + list(Path("Maestro_train").glob("**/*.midi"))
midi_paths_valid = list(Path("Maestro_valid").glob("**/*.mid")) + list(Path("Maestro_valid").glob("**/*.midi"))


Performing data augmentation: 100%|██████████| 1852/1852 [00:04<00:00, 406.03it/s]


tokenization

In [ ]:
def midi_valid(midi) -> bool:
    if any(ts.numerator != 4 for ts in midi.time_signature_changes):
        return False  # time signature different from 4/*, 4 beats per bar
    return True

if os.path.exists("tokenized"):
  shutil.rmtree("tokenized")

for dir in ("train", "valid"):
    tokenizer.tokenize_dataset(
        Path(f"Maestro_{dir}").resolve(),
        Path(f"tokenized_{dir}").resolve(),
        midi_valid,
    )

Tokenizing music files (content/tokenized_valid): 100%|██████████| 11162/11162 [04:41<00:00, 39.64it/s]


In [ ]:
!zip -r tokenized_train.zip tokenized_train
!zip -r tokenized_valid.zip tokenized_valid

Output streaming troncato alle ultime 5000 righe.
  adding: tokenized_valid/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav_7#d240.json (deflated 70%)
  adding: tokenized_valid/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav_0#v3.json (deflated 63%)
  adding: tokenized_valid/MIDI-Unprocessed_Schubert1-3_MID--AUDIO_02_R2_2018_wav_2#p12.json (deflated 65%)
  adding: tokenized_valid/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_15_R1_2013_wav--4_5#p12.json (deflated 65%)
  adding: tokenized_valid/MIDI-Unprocessed_16_R2_2009_01_ORIG_MID--AUDIO_16_R2_2009_16_R2_2009_01_WAV_4#v-3.json (deflated 64%)
  adding: tokenized_valid/MIDI-Unprocessed_XP_17_R2_2004_01_ORIG_MID--AUDIO_17_R2_2004_04_Track04_wav_8#d-240.json (deflated 65%)
  adding: tokenized_valid/MIDI-Unprocessed_046_PIANO046_MID--AUDIO-split_07-06-17_Piano-e_2-02_wav--1_3#v-3.json (deflated 62%)
  adding: tokenized_valid/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_03_R2_2013_wav--1_4#p-12.

Scaricare tokenizer addestrato e midi

In [4]:
!gdown --id 1VfmfvXp8QiH10Q5VFTc_TCyKvGSthD3-
!gdown --id 1Cnj-YhsNcj_GTGTAwjgRsBwK8fXXKZax

!unzip tokenized_train.zip
!unzip tokenized_valid.zip

Output streaming troncato alle ultime 5000 righe.
  inflating: tokenized_valid/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav_7#d240.json  
  inflating: tokenized_valid/MIDI-Unprocessed_01_R1_2006_01-09_ORIG_MID--AUDIO_01_R1_2006_04_Track04_wav_0#v3.json  
  inflating: tokenized_valid/MIDI-Unprocessed_Schubert1-3_MID--AUDIO_02_R2_2018_wav_2#p12.json  
  inflating: tokenized_valid/ORIG-MIDI_02_7_7_13_Group__MID--AUDIO_15_R1_2013_wav--4_5#p12.json  
  inflating: tokenized_valid/MIDI-Unprocessed_16_R2_2009_01_ORIG_MID--AUDIO_16_R2_2009_16_R2_2009_01_WAV_4#v-3.json  
  inflating: tokenized_valid/MIDI-Unprocessed_XP_17_R2_2004_01_ORIG_MID--AUDIO_17_R2_2004_04_Track04_wav_8#d-240.json  
  inflating: tokenized_valid/MIDI-Unprocessed_046_PIANO046_MID--AUDIO-split_07-06-17_Piano-e_2-02_wav--1_3#v-3.json  
  inflating: tokenized_valid/ORIG-MIDI_01_7_8_13_Group__MID--AUDIO_03_R2_2013_wav--1_4#p-12.json  
  inflating: tokenized_valid/MIDI-Unprocessed_060_PIANO060_M

In [5]:
!gdown --id 12der1lMHfMyMbGWImONehL_43AoECyzB

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=12der1lMHfMyMbGWImONehL_43AoECyzB
To: /content/tokenizer (2).json
100% 271k/271k [00:00<00:00, 124MB/s]


In [7]:
tokenizer = REMI(params="tokenizer (2).json")

In [8]:
def read_json(path: str) -> dict:
  with open(path, "r") as f:
    return json.load(f)

def read_json_files(json_file_paths):
    objects = []

    for file_path in tqdm(json_file_paths):
        try:
            objects.append(read_json(file_path))
        except FileNotFoundError:
            print(f"Error: File not found - {file_path}")
            return []
        except json.JSONDecodeError:
            print(f"Error decoding JSON in file: {file_path}")
            return []
    return objects

tokenized_train = list(Path("tokenized_train").resolve().glob("**/*.json"))
data_objects_train = read_json_files(tokenized_train)

tokenized_valid = list(Path("tokenized_valid").resolve().glob("**/*.json"))
data_objects_valid = read_json_files(tokenized_valid)



if data_objects_train and data_objects_valid:
    print(f"\nSuccessfully read {len(data_objects_train)} training JSON files.")
    print(f"Successfully read {len(data_objects_valid)} validation JSON files.")
else:
    print("Error reading JSON files.")

100%|██████████| 11162/11162 [00:01<00:00, 6836.35it/s]


Successfully read 51073 training JSON files.
Successfully read 11162 validation JSON files.


In [9]:
encoded_train = [np.array(song["ids"][0]) for song in data_objects_train]
encoded_valid = [np.array(song["ids"][0]) for song in data_objects_valid]
all_ids_train = np.concatenate(encoded_train)
all_ids_valid = np.concatenate(encoded_valid)

all_ids_train = all_ids_train.astype(dtype=np.int32)
all_ids_valid = all_ids_valid.astype(dtype=np.int32)

## GAN model

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_dim, hidden_dim, seq_length):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, hidden_dim * 8),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(hidden_dim * 8, hidden_dim * 4),
            nn.LayerNorm(hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(0.2),

            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.LayerNorm(hidden_dim * 2),
            nn.ReLU(),

            nn.Linear(hidden_dim * 2, seq_length),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)


Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, seq_length, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(seq_length, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),

            nn.Linear(hidden_dim * 2, hidden_dim * 4),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.2),

            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim * 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)


versione piu grande

In [10]:
class Generator(nn.Module):
    def __init__(self, noise_dim, hidden_dim, seq_length):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, hidden_dim * 16),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 16, hidden_dim * 8),
            nn.LayerNorm(hidden_dim * 8),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 8, hidden_dim * 4),
            nn.LayerNorm(hidden_dim * 4),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 4, hidden_dim * 2),
            nn.ReLU(),

            nn.Linear(hidden_dim * 2, seq_length),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)

In [11]:
class Discriminator(nn.Module):
    def __init__(self, seq_length, hidden_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(seq_length, hidden_dim * 2),
            nn.LeakyReLU(0.2),

            nn.Linear(hidden_dim * 2, hidden_dim * 4),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 4, hidden_dim * 8),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 8, hidden_dim * 16),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 16, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

In [12]:
vocab_size = len(tokenizer)
seq_length = 512
noise_dim = 512
hidden_dim = 1536
batch_size = 128

generator_config = {
    "noise_dim"  : noise_dim,
    "hidden_dim" : hidden_dim,
    "seq_length" : seq_length
}


discriminator_config = {
    "seq_length" : seq_length,
    "hidden_dim" : hidden_dim
}

generator = Generator(**generator_config)
discriminator = Discriminator(**discriminator_config)

gan_params = {
    "generator"                : generator,
    "discriminator"            : discriminator,
    "noise_dim"                : noise_dim,
    "seq_length"               : seq_length,
    "tokenizer"                : tokenizer,
}

GAN

In [13]:

class GAN:
    def __init__(
        self,
        generator,
        discriminator,
        noise_dim,
        seq_length,
        tokenizer,
        loss_fn=None,
        lr=5e-5,
        betas=(0.5, 0.999)
    ):
        self.generator = generator
        self.discriminator = discriminator
        self.noise_dim = noise_dim
        self.seq_length = seq_length
        self.tokenizer = tokenizer
        self.vocab_size = len(tokenizer)

        self.loss_fn = loss_fn or nn.BCEWithLogitsLoss()

        self.generator_optimizer = torch.optim.Adam(self.generator.parameters(), lr=lr, betas=betas)
        self.discriminator_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=lr, betas=betas)

        self._initialize_weights()

    @staticmethod
    def _init_weights(m):
        if isinstance(m, (nn.Linear, nn.Conv1d)):
            nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.zeros_(m.bias)

    def _initialize_weights(self):
        self.generator.apply(self._init_weights)
        self.discriminator.apply(self._init_weights)

    def noise(self, batch_size, device):
        return torch.randn(batch_size, self.noise_dim, device=device)

    def _generator_step(self, batch_size, device):
        self.generator.zero_grad()

        fake_noise = self.noise(batch_size, device)
        fake_data = self.generator(fake_noise)
        predictions = self.discriminator(fake_data)

        generator_loss = -torch.mean(predictions)

        if not torch.isnan(generator_loss) and not torch.isinf(generator_loss):
            generator_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.generator.parameters(), 1.0)
            self.generator_optimizer.step()
        else:
            print("[!] NaN or Inf detected in generator loss")

        return generator_loss

    def _discriminator_step(self, batch_size, real_data, device):
        self.discriminator.zero_grad()

        fake_noise = self.noise(batch_size, device)
        fake_data = self.generator(fake_noise).detach()

        real_preds = self.discriminator(real_data)
        fake_preds = self.discriminator(fake_data)

        real_labels = torch.ones_like(real_preds)
        fake_labels = torch.zeros_like(fake_preds)

        loss_real = self.loss_fn(real_preds, real_labels)
        loss_fake = self.loss_fn(fake_preds, fake_labels)

        discriminator_loss = loss_real + loss_fake

        if not torch.isnan(discriminator_loss) and not torch.isinf(discriminator_loss):
            discriminator_loss.backward()
            torch.nn.utils.clip_grad_norm_(self.discriminator.parameters(), 1.0)
            self.discriminator_optimizer.step()
        else:
            print("[!] NaN or Inf detected in discriminator loss")

        return discriminator_loss

    def _train_step(self, batch_size, real_data, device, gen_steps=1, disc_steps=1):
        disc_losses = [self._discriminator_step(batch_size, real_data, device) for _ in range(disc_steps)]
        gen_losses = [self._generator_step(batch_size, device) for _ in range(gen_steps)]

        avg_disc_loss = torch.stack(disc_losses).mean().item()
        avg_gen_loss = torch.stack(gen_losses).mean().item()
        return avg_disc_loss, avg_gen_loss

    def train(
        self,
        dataloader,
        epochs,
        device,
        loss_delta=0.7,
        steps_each_print=5,
        advantage_steps=2,
        alternate_training=False,
        gen_steps=1,
        disc_steps=1
    ):
        print(f"Starting training for {epochs} epochs...")

        self.generator.to(device).train()
        self.discriminator.to(device).train()

        discriminator_loss_history = []
        generator_loss_history = []

        for epoch in range(epochs):
            pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}")
            for real_data in pbar:
                real_data = real_data.to(device)
                batch_size = real_data.size(0)

                disc_loss, gen_loss = self._train_step(batch_size, real_data, device, gen_steps, disc_steps)

                discriminator_loss_history.append(disc_loss)
                generator_loss_history.append(gen_loss)

                if alternate_training:
                    if gen_loss - disc_loss > loss_delta:
                        gen_steps, disc_steps = advantage_steps, 1
                    elif disc_loss - gen_loss > loss_delta:
                        disc_steps, gen_steps = advantage_steps, 1
                    else:
                        gen_steps = disc_steps = 1

                if len(discriminator_loss_history) % steps_each_print == 0:
                    pbar.set_description(f"D Loss: {disc_loss:.4f}, G Loss: {gen_loss:.4f}")

            print(f"Epoch {epoch+1} - D Loss: {disc_loss:.4f}, G Loss: {gen_loss:.4f}")

        return discriminator_loss_history, generator_loss_history

    def save_generator(self, path):
        torch.save(self.generator.state_dict(), path)
        print(f"Generator saved to {path}")

    def save_discriminator(self, path):
        torch.save(self.discriminator.state_dict(), path)
        print(f"Discriminator saved to {path}")

    def predict(self, n_samples, device):
        z = self.noise(n_samples, device)
        return self.generator(z).cpu().detach().numpy()

    def generate(self, n_samples, base_output_name, device):
        predictions = self.predict(n_samples, device)
        boundary = int(self.vocab_size / 2)
        predictions = [x * boundary + boundary for x in predictions]

        for i, pred in enumerate(predictions):
            tokens = np.clip(np.round(pred).astype(int), 0, self.vocab_size - 1)
            try:
                decoded = self.tokenizer.decode([tokens])
                if hasattr(decoded, 'dump_midi'):
                    decoded.dump_midi(f"{base_output_name}_{i}.mid")
                else:
                    print(f"[!] Output {i} is not a MIDI-compatible object.")
            except Exception as e:
                print(f"[!] Error generating MIDI {i}: {e}")


In [14]:
class TokenDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return torch.tensor(self.data[idx], dtype=torch.float32)

vocab_size = len(tokenizer)

def normalize_data(data, vocab_size, interval=(-1, 1)):
    min_val, max_val = interval
    normalized_data = (data - vocab_size / 2) / (vocab_size / 2)
    scaled_data = normalized_data * (max_val - min_val) / 2 + (max_val + min_val) / 2

    return scaled_data

normalized_seq = normalize_data(all_ids_train, vocab_size)
print(f"Max is {normalized_seq.max()}, Min is {normalized_seq.min()}")

all_ids_train_seq = [normalized_seq[i:i + seq_length]
                 for i in range(0, len(normalized_seq) - seq_length, seq_length)]

dataset = TokenDataset(all_ids_train_seq)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(0)
_ = torch.ones(1, device='cuda')

Max is 0.9996, Min is -0.9988


In [15]:
gan = GAN(**gan_params)

training_arguments = {
    "dataloader"         : dataloader,
    "epochs"             : 20,
    "device"             : device,
    "steps_each_print"   : 10,
    "gen_steps"          : 1,
    "disc_steps"         : 1,
}

losses_discriminator, losses_generator = gan.train(**training_arguments)

Starting training for 20 epochs...


D Loss: 1.3863, G Loss: -0.0000: 100%|██████████| 786/786 [01:56<00:00,  6.75it/s]


Epoch 1 - D Loss: 1.3863, G Loss: -0.0000


D Loss: 1.6265, G Loss: -1.0000: 100%|██████████| 786/786 [01:55<00:00,  6.80it/s]


Epoch 2 - D Loss: 1.6265, G Loss: -1.0000


D Loss: 1.6265, G Loss: -1.0000: 100%|██████████| 786/786 [01:55<00:00,  6.80it/s]


Epoch 3 - D Loss: 1.6265, G Loss: -1.0000


D Loss: 1.6265, G Loss: -1.0000:   7%|▋         | 55/786 [00:08<01:51,  6.58it/s]


KeyboardInterrupt: 

In [ ]:
# Salva solo i pesi del generator e del discriminator
torch.save(generator.state_dict(), "generator_weights.pth")
torch.save(discriminator.state_dict(), "discriminator_weights.pth")


per caricare

In [ ]:
generator = Generator(**generator_config)
generator.load_state_dict(torch.load("generator_weights.pth"))
generator.eval()

discriminator = Discriminator(**discriminator_config)
discriminator.load_state_dict(torch.load("discriminator_weights.pth"))
discriminator.eval()


generate music

In [ ]:
def generate_noise(batch_size, noise_dim, device):
    """Genera rumore casuale per il generatore."""
    return torch.randn(batch_size, noise_dim, device=device)

def generate_predictions(generator, n_samples, noise_dim, device):
    """Genera output normalizzato dal generatore."""
    generator.eval()
    with torch.no_grad():
        z = generate_noise(n_samples, noise_dim, device)
        predictions = generator(z).cpu().numpy()
    return predictions

def decode_and_save(predictions, tokenizer, base_output_name, folder="generated", verbose=True):
    """Decodifica e salva i file MIDI generati."""
    os.makedirs(folder, exist_ok=True)
    vocab_size = len(tokenizer)
    boundary = vocab_size // 2

    for i, pred in enumerate(predictions):
        # Normalizza
        pred = (pred * boundary + boundary).astype(np.int32)
        pred[pred == vocab_size] = 0  # Clip to vocab

        try:
            midi = tokenizer.decode([pred])
            path = os.path.join(folder, f"{base_output_name}_{i}.mid")
            midi.dump_midi(path)
            if verbose:
                print(f"✔️ Saved: {path}")
        except Exception as e:
            print(f"Error decoding sample {i}: {e}")

def generate_midi_samples(generator, tokenizer, n_samples, noise_dim, device, base_output_name="generated/generated_midi", verbose=True):
    """Pipeline completa per generare file MIDI da un generatore."""
    predictions = generate_predictions(generator, n_samples, noise_dim, device)
    decode_and_save(predictions, tokenizer, base_output_name=base_output_name.split("/")[-1], folder=os.path.dirname(base_output_name), verbose=verbose)

def preview_sample(path_to_midi):
    """Mostra un'anteprima audio del file MIDI generato (solo per i notebook)."""
    try:
        display(Audio(path_to_midi))
    except Exception as e:
        print(f"Impossibile riprodurre audio: {e}")


In [ ]:
n_generations = 100
noise_dim = 512  # deve combaciare con quello del generatore
base_name = "generated/generated_midi"

# Genera e salva 100 MIDI
generate_midi_samples(generator, tokenizer, n_samples=n_generations, noise_dim=noise_dim, device=device, base_output_name=base_name)

# Mostra un esempio (es. il 10° file)
preview_sample(f"{base_name}_10.mid")

✔️ Saved: generated/generated_midi_0.mid
✔️ Saved: generated/generated_midi_1.mid
✔️ Saved: generated/generated_midi_2.mid
✔️ Saved: generated/generated_midi_3.mid
✔️ Saved: generated/generated_midi_4.mid
✔️ Saved: generated/generated_midi_5.mid
✔️ Saved: generated/generated_midi_6.mid
✔️ Saved: generated/generated_midi_7.mid
✔️ Saved: generated/generated_midi_8.mid
✔️ Saved: generated/generated_midi_9.mid
✔️ Saved: generated/generated_midi_10.mid
✔️ Saved: generated/generated_midi_11.mid
✔️ Saved: generated/generated_midi_12.mid
✔️ Saved: generated/generated_midi_13.mid
✔️ Saved: generated/generated_midi_14.mid
✔️ Saved: generated/generated_midi_15.mid
✔️ Saved: generated/generated_midi_16.mid
✔️ Saved: generated/generated_midi_17.mid
✔️ Saved: generated/generated_midi_18.mid
✔️ Saved: generated/generated_midi_19.mid
✔️ Saved: generated/generated_midi_20.mid
✔️ Saved: generated/generated_midi_21.mid
✔️ Saved: generated/generated_midi_22.mid
✔️ Saved: generated/generated_midi_23.mid
✔️

In [ ]:
!zip -r generetedMIDI.zip generated

  adding: generated/ (stored 0%)
  adding: generated/generated_midi_53.mid (deflated 45%)
  adding: generated/generated_midi_58.mid (deflated 45%)
  adding: generated/generated_midi_44.mid (deflated 44%)
  adding: generated/generated_midi_25.mid (deflated 46%)
  adding: generated/generated_midi_50.mid (deflated 45%)
  adding: generated/generated_midi_49.mid (deflated 45%)
  adding: generated/generated_midi_83.mid (deflated 44%)
  adding: generated/generated_midi_72.mid (deflated 44%)
  adding: generated/generated_midi_75.mid (deflated 44%)
  adding: generated/generated_midi_97.mid (deflated 45%)
  adding: generated/generated_midi_42.mid (deflated 44%)
  adding: generated/generated_midi_81.mid (deflated 45%)
  adding: generated/generated_midi_13.mid (deflated 44%)
  adding: generated/generated_midi_78.mid (deflated 46%)
  adding: generated/generated_midi_6.mid (deflated 46%)
  adding: generated/generated_midi_18.mid (deflated 44%)
  adding: generated/generated_midi_80.mid (deflated 45%)

VALUTARE prestazioni

In [ ]:
generated_midis = list(glob.glob("generated/*.mid"))

In [ ]:
def pitch_count(notes):
    """
    notes: list of int (pitch values)
    """
    return len(notes)


def note_density(notes, total_duration_sec):
    """
    notes: list of notes
    total_duration_sec: duration of piece in seconds
    """
    return len(notes) / total_duration_sec if total_duration_sec > 0 else 0

from collections import Counter

def rote_memorization_frequency(notes, pattern_length=3):
    """
    Return average number of repetitions for note patterns of length `pattern_length`
    """
    patterns = [tuple(notes[i:i+pattern_length]) for i in range(len(notes)-pattern_length+1)]
    counts = Counter(patterns)
    repeated = [count for pattern, count in counts.items() if count > 1]
    return np.mean(repeated) if repeated else 0

def pitch_intervals(notes):
    """
    Returns average absolute pitch interval between consecutive notes
    """
    if len(notes) < 2:
        return 0
    intervals = [abs(notes[i] - notes[i-1]) for i in range(1, len(notes))]
    return np.mean(intervals)

def pitch_class_histogram(notes):
    """
    Returns normalized histogram over 12 pitch classes
    """
    hist = np.zeros(12)
    for pitch in notes:
        hist[pitch % 12] += 1
    total = np.sum(hist)
    return hist / total if total > 0 else hist

from scipy.special import rel_entr

def kld(p, q):
    """
    Kullback-Leibler Divergence between distributions p and q
    p, q: numpy arrays representing probability distributions
    """
    p = np.asarray(p, dtype=np.float64) + 1e-12  # avoid zeros
    q = np.asarray(q, dtype=np.float64) + 1e-12
    p /= p.sum()
    q /= q.sum()
    return np.sum(rel_entr(p, q))


In [ ]:
def compute_all_metrics(generated_notes, real_notes=None, total_duration_sec=30.0):
    metrics = {
        "Pitch Count": pitch_count(generated_notes),
        "Note Density": note_density(generated_notes, total_duration_sec),
        "Rote Memorization": rote_memorization_frequency(generated_notes),
        "Pitch Interval": pitch_intervals(generated_notes),
        "Pitch Class Histogram": pitch_class_histogram(generated_notes),
    }

    if real_notes is not None:
        metrics["PCH KLD"] = kld(
            metrics["Pitch Class Histogram"],
            pitch_class_histogram(real_notes)
        )

    return metrics


In [ ]:
metrics = compute_all_metrics(generated_notes, real_notes)

for name, value in metrics.items():
    if isinstance(value, np.ndarray):
        print(f"{name}: {value.round(3)}")
    else:
        print(f"{name}: {value:.3f}")
